In [1]:
!git clone https://github.com/songhuadan/xai_transformers.git

Cloning into 'xai_transformers'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 45 (delta 12), reused 29 (delta 7), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [2]:
import os
os.chdir(f'./xai_transformers')

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import os
import sys
import inspect

In [5]:
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 43.2 MB/s 
     |████████████████████████████████| 182 kB 49.0 MB/s 


In [9]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 4.8 MB/s 
     |████████████████████████████████| 213 kB 68.6 MB/s 
     |████████████████████████████████| 132 kB 33.6 MB/s 
     |████████████████████████████████| 127 kB 54.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [6]:
from xai_transformer import make_p_layer, BertPooler, BertAttention
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from sst import get_sst_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from attribution import softmax, compute_joint_attention, get_flow_relevance_for_all_layers , _compute_rollout_attention
from utils import flip, set_up_dir
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import matplotlib.pyplot as plt
import pickle

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


In [7]:
RANDOM_SEED = 42


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--textattack--bert-base-uncased-SST-2/snapshots/95f0f6f859b35c8ff0863ae3cd4e2dbc702c0ae2/config.json
Model config BertConfig {
  "_name_or_path": "textattack/bert-base-uncased-SST-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--textattack--bert-base-uncased-SST-2/snapshots/95f0f6f859b35c8ff0863ae3cd4e2dbc702c0ae2/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--textattack--bert-base-uncased-SST-2/snapshots/95f0f6f859b35c8ff0863ae3cd4e2dbc702c0ae2/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--textattack--bert-base-uncased-SST-2/snapshots/95f0f6f859b35c8ff0863ae3cd4e2dbc702c0ae2/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--textattack--bert-base-uncased-SST-2/snapshots/95f0f6f859b35c8ff0863ae3cd4e2dbc702c0ae2/config.json
Model config BertConfig {
  "_name_or_path": "textattack/bert-base-uncased-SST-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "a

In [12]:
# Load data
datasets  = load_dataset("glue", 'sst2')


label_to_id = {v: i for i, v in enumerate([0,1])}

_, test_data_loader = get_sst_dataset(datasets, tokenizer)

  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
# Init Model
class Config(object):
    
    def __init__(self):
        self.hidden_size = 768
        self.num_attention_heads = 12
        self.layer_norm_eps = 1e-12
        self.n_classes = 2
        self.n_blocks = 3
                    
        self.attention_head_size = int(self.hidden_size / self.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size
        
        self.detach_layernorm = True # Detaches the attention-block-output LayerNorm
        self.detach_kq = True # Detaches the kq-softmax branch
        self.device = device
        self.train_mode = False
        self.detach_mean = True #

        

bert_model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")
bert_model.bert.embeddings.requires_grad = False
for name, param in bert_model.named_parameters():                
    if name.startswith('embeddings'):
        param.requires_grad = False
        
pretrained_embeds = bert_model.bert.embeddings

params = torch.load('SST/sst2-3layer-model.pt', map_location=torch.device(device))


def rename_params(key):
    for k_ in ['key','query', 'value']:
        key=key.replace(k_, 'p'+k_)
    return key

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--textattack--bert-base-uncased-SST-2/snapshots/95f0f6f859b35c8ff0863ae3cd4e2dbc702c0ae2/config.json
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--textattack--bert-base-uncased-SST-2/snapshots/95f0f6f859b35c8ff0863ae3cd4e2dbc702c0ae2/pyto

In [19]:
# Transformer Model 
config = Config()
config.detach_layernorm = False # Detaches the attention-block-output LayerNorm
config.detach_kq = False
model = BertAttention(config, pretrained_embeds)
model.load_state_dict(params)
model.to(device)
models = {'none': model}

# Transformer Model 
print('Detach SM')
config = Config()
config.detach_layernorm = False # Detaches the attention-block-output LayerNorm
config.detach_kq = True
model = BertAttention(config, pretrained_embeds)
model.load_state_dict(params)
model.to(device)
models['detach_KQ'] = model

Detach SM
Detach K-Q-softmax branch
Detach K-Q-softmax branch
Detach K-Q-softmax branch


In [20]:
print('Detach SM +  LN')
# Transformer Model 
config = Config()
model = BertAttention(config, pretrained_embeds)
model.load_state_dict(params)
model.to(device)
models['detach_KQ_LNorm'] = model

Detach SM +  LN
Detach LayerNorm Mean+Norm
Detach K-Q-softmax branch
Detach LayerNorm Mean+Norm
Detach K-Q-softmax branch
Detach LayerNorm Mean+Norm
Detach K-Q-softmax branch
Detach LayerNorm Mean+Norm


In [21]:
print('Detach SM +  LN without mean')
# Transformer Model 
config = Config()
config.detach_layernorm = True # Detaches the attention-block-output LayerNorm
config.detach_mean = False # Detaches the attention-block-output LayerNorm
config.detach_kq = True
model = BertAttention(config, pretrained_embeds)
model.load_state_dict(params)
model.to(device)
models['detach_KQ_LNorm_Norm'] = model

Detach SM +  LN without mean
Detach LayerNorm only Norm
Detach K-Q-softmax branch
Detach LayerNorm only Norm
Detach K-Q-softmax branch
Detach LayerNorm only Norm
Detach K-Q-softmax branch
Detach LayerNorm only Norm


In [22]:
# Run Flipping
UNK_token = tokenizer.unk_token_id

fracs = np.linspace(0.,1.,11)

In [ ]:
# Uncomment cases if they are not computed already in all_flips
gammas = [0.00,0.00, 0.00]
for flip_case in [ 'generate' , 'pruning']:
    print(flip_case)
    all_flips = {}
    
    res_conservation = {}
    for case, random_order in [('random', True), 
                               ('attn_last', False),  
                               ('rollout_2', False),
                               ('GAE' , False),
                               ('gi', False),
                               ('lrp_detach_KQ', False),
                               ('lrp_detach_KQ_LNorm_Norm', False),
                              ]:
        print(case)
        layer_idxs = model.attention_probs.keys()
        M,E, EVOLUTION = [],[], []
        C = []
        j=0
        if case in ['gi', 'lrp', 'GAE']:
            model = models['none']
        elif case in ['gi_detach_KQ', 'lrp_detach_KQ']:
            model = models['detach_KQ']
        elif case in ['detach_KQ_LNorm_Norm', 'lrp_detach_KQ_LNorm_Norm']   : 
            model = models['detach_KQ_LNorm_Norm']
        else:
            model = models['detach_KQ_LNorm']
        for x in test_data_loader:
            input_ids = torch.tensor(np.float32(x['input_ids']) , requires_grad=True).unsqueeze(0).long().to(device)
            attention_mask = torch.tensor(np.float32(x['attention_mask']), requires_grad=True).unsqueeze(0).long().to(device)
            token_type_ids = torch.tensor(np.float32(x['token_type_ids'])).unsqueeze(0).long().to(device)
            words = tokenizer.convert_ids_to_tokens(input_ids.squeeze())
            y_true = torch.tensor(x['label']).to(device)
            labels_in = torch.tensor([int(y_true)]*len(input_ids)).long().to(device)
            if case == 'GAE':
                outs = model.forward_and_explain(input_ids=input_ids, cl=y_true,
                 labels = labels_in , method=case)
            else:
                outs = model(input_ids=input_ids,
                             labels = labels_in)
            loss = outs['loss'].detach().cpu().numpy()
            y_pred = np.argmax(outs['logits'].squeeze().detach().cpu().numpy())
            if case =='random':
                attribution = np.random.normal(0,1, np.array(words).shape) #np.random.normal(np.array(words).shape)

            elif case == 'attn_last':
                attribution = np.mean([x_.sum(0) for x_ in model.attention_probs[max(layer_idxs)].detach().cpu().numpy()[0]],0)
            elif 'rollout' in case:
                attns = [model.attention_probs[k].detach().cpu().numpy() for k in sorted(model.attention_probs.keys())]
                attentions_mat = np.stack(attns,axis=0).squeeze()
                res_att_mat = attentions_mat.sum(axis=1)/attentions_mat.shape[1]
                joint_attentions = compute_joint_attention(res_att_mat, add_residual=True)
                idx = int(case.replace('rollout_',''))
                attribution = joint_attentions[idx].sum(0)
            elif 'GAE' in case:
                attns = [model.attention_probs[k].detach().cpu().numpy() for k in sorted(model.attention_probs.keys())]
                attentions_mat = np.stack(attns,axis=0).squeeze()
                attns = [model.attention_gradients[k].detach().cpu().numpy() for k in sorted(model.attention_gradients.keys())]
                attentions_grads = np.stack(attns,axis=0).squeeze()
                attentions_mat = torch.tensor(attentions_mat * attentions_grads).clamp(min=0)
                attentions_mat = torch.tensor(attentions_mat).clamp(min=0).mean(dim=1)
                joint_attentions = _compute_rollout_attention(attentions_mat)
                joint_attentions[:, 0, 0] = 0
                idx = 0
                attribution = joint_attentions[idx].sum(0)
            elif 'attention_flow' in case:
                attns = [model.attention_probs[k].detach().cpu().numpy() for k in sorted(model.attention_probs.keys())]
                attentions_mat = np.stack(attns,axis=0).squeeze()
                idx = int(case.replace('attention_flow_',''))

                aflow = get_flow_relevance_for_all_layers(input_ids.detach().cpu().numpy().squeeze(),  
                                                          attentions_mat[:, np.newaxis],
                                                          tokens=words, 
                                                          layers=[idx],
                                                          pad_token= tokenizer.pad_token_id)
                assert len(aflow) == 1
                attribution =  aflow[0]
            elif case == 'naive_gi':
                outs = model.forward_and_explain(input_ids=input_ids, cl=y_true,
                               labels = labels_in,
                                 gammas = [0.0,0.0,0.0])

                attribution = outs['R'].squeeze()


            elif case in ['gi', 'gi_detach_KQ_LNorm', 'gi_detach_KQ', 'gi_detach_KQ_LNorm_Norm']:
                outs = model.forward_and_explain(input_ids=input_ids, cl=y_true,
                               labels = labels_in,
                               gammas = [0.0,0.0,0.0])

                attribution = outs['R'].squeeze()


            elif case in ['lrp', 'lrp_detach_KQ_LNorm', 'lrp_detach_KQ', 'lrp_detach_KQ_LNorm_Norm']:

                outs = model.forward_and_explain(input_ids=input_ids, cl=y_true,
                               labels = labels_in, 
                                gammas = gammas)


                attribution = outs['R'].squeeze()


            if j==0:
                print(type(attribution), type(input_ids),  type(words))

            m, e, evolution = flip(model,
                              x=attribution, 
                             token_ids=input_ids, 
                             tokens=words,
                             y_true=y_true, 
                             fracs=fracs, 
                             flip_case=flip_case,
                             random_order = random_order, 
                             tokenizer=tokenizer,
                             device=device)


            M.append(m)
            E.append(e)
            EVOLUTION.append(evolution)
            

            if j%500==0:
                print('****',j)


            j+=1
        all_flips[case]= {'E':E, 'M':M, 'Evolution':EVOLUTION} 

        res_conservation[case] = C

    f, axs = plt.subplots(1, 2, figsize=(14, 8))
    for k, v in all_flips.items():
        print(len(v['M']))
        axs[0].plot(np.nanmean(v['M'], axis=0), label=k)
        axs[0].set_title('($y_0$-$y_p$)$^2$')
        axs[1].plot(np.nanmean(v['E'], axis=0), label=k)
        axs[1].set_title('logits$_k$')    
    plt.legend()
    
    f.savefig('{}.png'.format(flip_case), dpi=300)
    pickle.dump(all_flips, open('all_flips_{}_sst.p'.format(flip_case), 'wb'))


generate
random
<class 'numpy.ndarray'> <class 'torch.Tensor'> <class 'list'>


/content/xai_transformers/utils.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs0 = torch.tensor(token_ids).to(device)


**** 0
**** 500
attn_last
<class 'numpy.ndarray'> <class 'torch.Tensor'> <class 'list'>
**** 0
**** 500
rollout_2
<class 'numpy.ndarray'> <class 'torch.Tensor'> <class 'list'>
**** 0
